## URL Matching
We observe that image url  and text url pairs may have some explicit relationships. More specifically, an image url and a text url may be related if they contain common keywords. 
- tokenize both image urls and text urls
- remove stop words from the tokens
- an image url and a text url is considered to be matched if they one common token
- The ranking of images is sorted by the number of total matched.

In [ ]:
img_id_name_dict={}
with open('MediaEvalNewsImagesBatch04images.tsv') as file:
    next(file)
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    for line in lines:
        segs=line.split("\t")
        if len(segs)<3:
            break
        img_id=segs[1]
        img_name_full=segs[0].split("/")
        img_name=img_name_full[len(img_name_full)-1]
        tokens=img_name.split(".")[0].split("-")
        tokens = [item for item in tokens if item.isalpha() and item!="null"]
        img_id_name_dict[img_id]=tokens

In [ ]:
print(len(img_id_name_dict))

In [ ]:
with open('MediaEvalNewsImagesBatch04articles.tsv', encoding="utf8") as file:
    lines = file.readlines()
    print(lines[0])
    print(lines[1].split("\t")[2])

In [ ]:
art_id_title_dict={}
with open('MediaEvalNewsImagesBatch04articles.tsv', encoding="utf8") as file:
    next(file)
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    for line in lines:
        segs=line.split("\t")
        if len(segs)<3:
            break
        ar_id=segs[0]
        ar_name_full=segs[6].split(" ")
        ar_name_full = [item for item in ar_name_full if item.isalpha() and item!="null"]
        art_id_title_dict[ar_id]=ar_name_full

In [ ]:
art_id_name_dict={}
with open('MediaEvalNewsImagesBatch04articles.tsv', encoding="utf8") as file:
    next(file)
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    for line in lines:
        segs=line.split("\t")
        if len(segs)<3:
            break
        ar_id=segs[0]
        ar_name_full=segs[2].split("/")
        ar_name=ar_name_full[len(ar_name_full)-1]
        tokens=ar_name.split(".")[0].split("-")
        tokens = [item for item in tokens if item.isalpha() and item!="null"]
        art_id_name_dict[ar_id]=tokens

In [ ]:
candidates={}
filter = ['in', 'der', 'die', 'und', 'im', 'auf', 'mit', 'fuer', 'von', 'den', 'an', 'fc', 'das', 'am', 'vor', 'aus', 'dem', 'anfang', 'sich', 'bei', 'ein', 'des', 'zu', 'sind', 'eine', 'ueber', 'gegen', 'nach', 'ist']
filter+=['zum','beim',
'wird',
'nrw',
'nicht',
'als',
'mehr',
'ab',
'zur',
'werden',
'hat',
's',
'wie',
'einem',
'auch',
'e',
'unter',
'wieder',
'vom',
'so',
'um',
'noch',
'will',
'afd',
'war',
'strasse']
def orig_relation(art_id_name_dict, img_id_name_dict):
    total=0
    result={}
    for art_k, art_v in art_id_name_dict.items():
        cnt=0
        flag=False
        
        for img_k, img_v in img_id_name_dict.items():
            common_elements = [x for x in art_v if x in img_v and x not in filter and len(x)>1 ]
            if len(common_elements)>0:
                if art_k not in result:
                    result[art_k]=[]
                result[art_k].append((img_k, len(common_elements)))
                flag=True
                cnt+=1
                for ele in common_elements:
                    if ele not in candidates:
                        candidates[ele]=0
                    candidates[ele]+=1
        if flag:
            total+=1
    return result

def relation(art_id_name_dict, img_id_name_dict):
    total=0
    result={}
    for art_k, art_v in art_id_name_dict.items():
        cnt=0
        flag=False
        
        for img_k, img_v in img_id_name_dict.items():
            common_elements = [x for x in art_v if x in img_v and x not in filter and len(x)>1 ]
            if len(common_elements)>0:
                print(art_k)
                if art_k not in result:
                    result[art_k]=[]
                result[art_k].append((img_k, len(common_elements)))
                print(img_k, common_elements)
                flag=True
                cnt+=1
                for ele in common_elements:
                    if ele not in candidates:
                        candidates[ele]=0
                    candidates[ele]+=1
        if art_k in result:
            temp_list=result[art_k]
            temp_list.sort(key=lambda x:x[1], reverse=True)
            result[art_k]=[i[0] for i in temp_list]
            print(result[art_k])
        print(cnt)
        if flag:
            total+=1
    print(total, " matches")
    return result

In [ ]:
result = relation(art_id_name_dict, img_id_name_dict)

In [ ]:
print(result)
print(len(result['1001935289']))
#result1 = relation(art_id_title_dict, img_id_name_dict)

In [ ]:
submission={}
with open('submission_txstate_firewheel.tsv') as file:
    next(file)
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    for line in lines:
        segs=line.split("\t")
        submission[segs[0]]=segs[1:]
        print(len(submission[segs[0]]))

In [ ]:
print(submission)

In [ ]:
cnt=0
new_submission={}
for s_id, s_value in submission.items():
    if s_id in result:
        diff_elements = [x for x in result[s_id] if x not in s_value ]
        common_elements= [x for x in s_value if x in result[s_id] ]
        tail_elements=s_value[len(s_value)-len(diff_elements):]
        common_ele_in_tail=[x for x in common_elements if x in tail_elements]
        if len(common_ele_in_tail)>0:
            new_value=diff_elements+common_ele_in_tail+s_value[:len(s_value)-len(diff_elements)-len(common_ele_in_tail)]
        else:
            new_value=diff_elements+s_value[:len(s_value)-len(diff_elements)]
        new_submission[s_id]= new_value

    else:
        new_submission[s_id]=s_value

In [ ]:
print((result['1369955288']))
print(len(new_submission['1369955288']))

In [ ]:
for new_id, new_value in new_submission.items():
    if new_id in result:
        diff_elements = [x for x in result[new_id] if x not in new_value ]
        if len(diff_elements)>0:
            print(diff_elements[0] in result)
            print(new_id)
            print(diff_elements)

In [ ]:
print ('136523' in new_submission['1369955288'])
print ('135231' in result['1058395241'])

In [ ]:
with open('3rd_submission_txstate_firewheel.tsv', 'a') as the_file:
    header="particleID"
    for i in range(100):
        header+="\t"+"iid"+str(i+1)
    the_file.write(header+"\n")
    for key, value in new_submission.items():
        the_file.write(key+'\t'+ "\t".join(value)+"\n")